# Uncertainty quantification and Polynomial Chaos Expansion

## Exercise 1

We consider the primal formulation of the Darcy problem: the only unknown is the pressure $p$ and the numerical scheme we are considering is the TPFA.

Let $\Omega = [0, 1]^2$ be the domain of interest with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$f = 0$ the source term, we want to solve the following problem: find $p$ such that
$$
\nabla \cdot \left(- \frac{k}{\mu} \nabla p\right) = 0
\quad \text{in } \Omega
$$
with boundary conditions:
$$ p(x,y) = 1 - y \text{ on } \partial \Omega$$

We consider the following relation, called Kozeny-Carman, between the permeability $k$ in $[m]^2$ and the porosity $\phi$ in $[\cdot]$ of the media
$$
k = k_{ref} \frac{\phi^3}{ (1-\phi)^2} \frac{(1-\phi_{ref})^2}{\phi_{ref}^3}
$$
where $k_{ref}$ in $[m]^2$ is a reference value for the media permeability and $\phi_{ref}$ in $[\cdot]$ a reference value for the porosity.
Moreover, we assume that the value of the porosity is then modeled based on the coefficient of grain uniformity $\eta$ in $[\cdot]$, read from a file, which is given as
$$
\phi = \phi_{ref} 10^\eta
$$
We consider affected by uncertainty the reference values with distributions $k_{ref} = 10^{-2}$, $\phi_{ref} \sim U(0.125, 0.675)$.
The liquid viscosity $\mu$ in $[kg \, s^{-1} \, m^{-1}]$ is also affected by uncertainty due to its dependency on the temperature, we assume
$$
\mu = \mu_{ref} e^{-T_{ratio}}
$$
with $T_{ratio} = T_{ref} / T \sim U(1.5, 2)$ and $\mu_{ref} = 1$ for simplicity.


The reference articoles are the following [[1]](https://www.mdpi.com/2073-4441/14/4/546) and [[2]](https://doi.org/10.1016/j.matcom.2024.10.024).

To run the code it is important to install the package [chaospy](https://chaospy.readthedocs.io/en/master/), for doing so open a terminal in the container and with the environment active type
`pip install chaospy`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp

import porepy as pp

from solve_problem import *

We create now the grid, which could be: Cartesian, simplicial, Voronoi, or octagonal.

In [ ]:
# load the porosity from file
file_name = "porosity"
phi_exp = np.loadtxt(file_name, skiprows=1)[:, 2]

mu_ref = 1
k_ref = 1e-2

phi = lambda phi_ref: phi_ref * np.power(10, phi_exp)
mu = lambda t_ratio: mu_ref * np.exp(-t_ratio)

seed = 42

In [ ]:
dim = 2
num = int(np.sqrt(phi_exp.size))

# creation of a Cartesian grid
sd = pp.CartGrid([num] * dim, [1] * dim)

# compute the geometrical properties of the grid
sd.compute_geometry()

In [ ]:
perm = (
    lambda phi_ref: k_ref
    * np.power(phi(phi_ref), 3)
    * np.power(1 - phi_ref, 2)
    / (np.power(phi_ref, 3) * np.power(1 - phi(phi_ref), 2))
)
fom = lambda x: solve_problem(sd, perm(x[0]) / mu(x[1]))[1]

In [ ]:
# define the uncertain parameters
phi_ref = cp.Uniform(lower=0.125, upper=0.675)
t_ratio = cp.Uniform(lower=1.5, upper=2)
joint_dist = cp.J(phi_ref, t_ratio)

# generate the sparse grid nodes for the distributions
level = 2
sparse_nodes, sparse_weights = cp.generate_quadrature(level, joint_dist, sparse=True)
print("Number of samples", sparse_nodes.shape[1])

# compute the FOM evaluations, it might take a while
fom_eval = np.array([fom(s) for s in sparse_nodes.T])

In [ ]:
plt.scatter(*sparse_nodes, s=50)
# plt.grid(color="gray", linestyle="--", linewidth=0.7, alpha=0.5)
# plt.savefig("sparse_grid_level2.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# define ROM with orthogonal polynomials
np.bool = np.bool_  # to fix chaospy with the current version of python
order = 5
expansion = cp.generate_expansion(order, dist=joint_dist)

# generate the ROM based on the FOM evaluations
rom = cp.fit_regression(expansion, sparse_nodes, fom_eval)

# compare the FOM and ROM evaluations for a given set of parameters
vals = np.array([0.35, 3])
print(rom(*vals))
print(fom(vals))

# compute the mean, standard deviation and sensitivity indices
mean = cp.E(rom, joint_dist)
std = cp.Std(rom, joint_dist)
sens = cp.Sens_m(rom, joint_dist)

print("Mean, standard deviation and sensitivity indices:", mean, std, sens)

In [ ]:
rom_num_samples = 1000
rom_samples = joint_dist.sample(rom_num_samples, rule="latin_hypercube", seed=seed)
rom_eval = np.array([rom(*s) for s in rom_samples.T])

We export the solutions to be visualized by [ParaView](https://www.paraview.org/).

In [ ]:
plt.hist(rom_eval, 30)
plt.show()

In [ ]:
phi_ref.lower

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

N = 100
x = np.linspace(phi_ref.lower, phi_ref.upper, N)
y = np.linspace(t_ratio.lower, t_ratio.upper, N)

X, Y = np.meshgrid(x, y)
zs = np.array([rom(x, y) for x, y in zip(np.ravel(X), np.ravel(Y))])
Z = zs.reshape(X.shape)

ax.plot_surface(X, Y, Z)

ax.set_xlabel("X Label")
ax.set_ylabel("Y Label")
ax.set_zlabel("Z Label")

plt.show()